## Check Validity of Soduku Board 

In [2]:
import Inference as inf
def check_validity(line):
    data = np.array([int(j) for j in line]).reshape((9,9))
    inf.conflict_pairs={}
    inf.conflict_set=set()
    return inf.check_valid_board(data)

Using TensorFlow backend.


## Generate Random Non Valid Soduku Data

In [4]:
import random
import numpy as np

file = open("notvalid.txt", "w")

n = 81
for i in range(10000):
    line = ''.join(["{}".format(random.randint(1, 9)) for num in range(0, n)])
    valid = check_validity(line)
    #print(valid)
    if not valid:
        line = line + ',0\n'
        file.write(line)
file.close()

## Process Data for Combined Valid and Non Valid Data for training and testing

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
def get_data(): 

    data = pd.read_csv('Dataset/Combined.csv')

    feat_raw = data['Puzzle']
    label_raw = data['Valid']

    feat = []
    label = []

    for i in feat_raw:
    
        x = np.array([int(j) for j in i]).reshape((9,9,1))
        feat.append(x)
    
    feat = np.array(feat)
    feat = feat/9
    feat -= .5    
    
    '''for i in label_raw:
    
        x = np.array(label_raw[i])
        label.append(x)   '''
    
    label = np.array(label_raw)
    
    del(feat_raw)
    del(label_raw)    

    x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.5, random_state=42,shuffle=True)
    
    return x_train, x_test, y_train, y_test

In [7]:
x_train, x_test, y_train, y_test = get_data()
#print(f'{x_train[0]}----{y_train[0]}')

## Create Model for checking if Soduko board configuarion is Valid

In [16]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape

def get_model():

    model = keras.models.Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model

Using TensorFlow backend.


In [151]:
model = get_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 9, 9, 64)          640       
_________________________________________________________________
batch_normalization_11 (Batc (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_12 (Batc (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten_6 (Flatten)          (None, 10368)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)               

In [152]:
adam = keras.optimizers.adam(lr=.001)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
9902/9902 [==============================] - 15s 1ms/step - loss: 0.4329 - accuracy: 0.8850
Epoch 2/2
9902/9902 [==============================] - 13s 1ms/step - loss: 0.2568 - accuracy: 0.9966 1s - loss:


## Prediction and Confusion Matrix for test set

In [153]:
y_pred=model.predict(x_test)
y_pred =(y_pred>0.5)
print(y_pred)
print(y_test)

[[False]
 [ True]
 [False]
 ...
 [ True]
 [False]
 [False]]
[0 1 0 ... 1 0 0]


In [154]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[4944    7]
 [   0 4951]]


In [155]:
model.save('SodokuValidator.h5')

In [14]:
def process(feat_raw):
    
    feat = np.array([int(j) for j in feat_raw]).reshape((1,9,9,1))
    #feat.append(x)
    
    feat = np.array(feat)
    feat = feat/9
    feat -= .5   
    
    return feat

## Insert random ambiguities in valid dataset

In [218]:
test = '641874532239139687837625419163957248982416753475382196328741965519263874746598321'
data = process(test)
#print(data)
print(model.predict(data))

[[0.633241]]


In [8]:
import pandas as pd
import numpy as np
import random

#Inserts ambiguities at random positions
def create_ambiguous():
    data = pd.read_csv('Dataset/Valid.csv')
    feat = []
    random.seed(32)
    feat_raw = data['Puzzle']
    file1 = open("semilexicalras.txt", "w")
    file2 = open("semilexicalraslabel.txt", "w")
    for i in feat_raw:
        x = np.array([int(j) for j in i]).reshape((9,9))
        y = np.copy(x).tolist()
        #print(x)
        rowrange = random.randint(0,8)+1
        colrange = random.randint(0,8)+1
        #print(rowrange)
        #print(colrange)
        newdict = {}
        for i in range(rowrange):
            for j in range(colrange):
                if x[i][j] == 4:
                    x[i][j] = 0
                    y[i][j] = '9_4'
                    newdict[(i,j)] = {9:50,4:50}
                if x[i][j] == 9:
                    x[i][j] = 0
                    y[i][j] = '4_9'
                    newdict[(i,j)] = {9:50,4:50}
                if x[i][j] == 3:
                    x[i][j] = 0
                    y[i][j] = '5_3'
                    newdict[(i,j)] = {3:50,5:50}
                if x[i][j] == 5:
                    x[i][j] = 0
                    y[i][j] = '3_5'
                    newdict[(i,j)] = {3:50,5:50}
        str1= ''
        str2= ''
        for i in range(9):
            for j in range(9):
                str2=str2+str(y[i][j])
                str1=str1+str(x[i][j])
        line1 = str1 + ',1\n'
        file1.write(line1)
        line2 = str2 + '\n'
        file2.write(line2)
        #print(newdict)
        #print(x.tolist())
        #print(y)
        #print(str1)
        #print(str2)
    file1.close()
    file2.close()

In [9]:
create_ambiguous()

## Run end to end CNN model for semi-lexical tokens

In [23]:
fileSL = open("semilexicalras.txt", "r")
model = keras.models.load_model('SodokuValidator.h5')
sl_test = []
for line in fileSL:
    sl_test.append(line.split(',')[0])
feat = []
for i in sl_test:
    
    x = np.array([int(j) for j in i]).reshape((9,9,1))
    feat.append(x)
    
feat = np.array(feat)
feat = feat/9
feat -= .5

y_pred=model.predict(feat)
y_pred =(y_pred>0.5)
print(sum(y_pred))

[1030]


## Run Semi-Lexical Framework for board with semi-lexical tokens

In [99]:
fileSL = open("semilexicalras.txt", "r")
fileSLLaber = open("semilexicalraslabel.txt", "r")
sl_test = []
sl_label = []

for line in fileSL:
    sl_test.append(line.split(',')[0])
for line in fileSLLaber:
    boardlist1 = []
    boardlist2 = []
    for element in line:
        boardlist1.append(element)
    x = 0
    while (x<len(boardlist1)):
        if x < len(boardlist1)-1 and boardlist1[x+1] == '_':
            strx = boardlist1[x]+boardlist1[x+1]+boardlist1[x+2]
            boardlist2.append(strx)
            x  = x+3
        else:
            boardlist2.append(boardlist1[x])
            x=x+1
        
    sl_label.append(boardlist2)
print(len(boardlist1))
feat = []
for i in sl_test:  
    x = np.array([int(j) for j in i]).reshape((9,9))
    feat.append(x)
print(len(sl_label[0]))
print(sl_label[0])
print(feat[0])


84
82
['3_5', '1', '4_9', '6', '2', '8', '4', '7', '3', '8', '7', '5_3', '4_9', '4', '1', '6', '5', '2', '6', '4', '2', '3', '7', '5', '9', '1', '8', '9', '3', '4', '7', '1', '6', '2', '8', '5', '2', '5', '6', '4', '8', '3', '7', '9', '1', '1', '8', '7', '2', '5', '9', '3', '4', '6', '4', '2', '8', '5', '6', '7', '1', '3', '9', '7', '9', '5', '1', '3', '2', '8', '6', '4', '3', '6', '1', '8', '9', '4', '5', '2', '7', '\n']
[[0 1 0 6 2 8 4 7 3]
 [8 7 0 0 4 1 6 5 2]
 [6 4 2 3 7 5 9 1 8]
 [9 3 4 7 1 6 2 8 5]
 [2 5 6 4 8 3 7 9 1]
 [1 8 7 2 5 9 3 4 6]
 [4 2 8 5 6 7 1 3 9]
 [7 9 5 1 3 2 8 6 4]
 [3 6 1 8 9 4 5 2 7]]


In [117]:
semilexicalcell_predlist = []
count = 0
while count < len(feat):
    semilexicalpreddict = {}
    for i in range(9):
        for j in range(9):
            if feat[count][i][j] == 0:
                mapdict = {}
                predlist = sl_label[count][((i*9)+j)].split('_')
                mapdict[predlist[0]] = 50
                mapdict[predlist[1]] = 50
                semilexicalpreddict[(i,j)] = mapdict
    count = count + 1
    semilexicalcell_predlist.append(semilexicalpreddict)
print(feat[0])
#print(semilexicalcell_predlist[0])

[[3 1 4 6 2 8 4 7 3]
 [8 7 5 4 4 1 6 5 2]
 [6 4 2 3 7 5 9 1 8]
 [9 3 4 7 1 6 2 8 5]
 [2 5 6 4 8 3 7 9 1]
 [1 8 7 2 5 9 3 4 6]
 [4 2 8 5 6 7 1 3 9]
 [7 9 5 1 3 2 8 6 4]
 [3 6 1 8 9 4 5 2 7]]


In [116]:
import Inference as inf
number_count = 0
for i in range(len(feat)):
    val = inf.sd.call_solve_sudoku(feat[i],semilexicalcell_predlist[i])
    if(isinstance(val,np.ndarray)):
        number_count = number_count+1
print(number_count)

9804
